In [1]:
import os
os.chdir("..")

In [2]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable, Function
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.distributions import Normal

import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
import scipy.misc as scpm
import math
from datta.neural_decoding_models.base_model import BaseModel
import h5py
import copy

ModuleNotFoundError: No module named 'datta.neural_decoding_models.base_model'

In [3]:
batch_size=1000

In [6]:
#### LOAD PCA DATA ####
n_mice=17
h5_temp = h5py.File('whitened_clean_pca.h5','r')

In [11]:
for k in h5_temp.keys():
    print(k)

3382c9d2-19a2-4b91-83c5-e1f0d1ead9f1
5cb78054-857e-4e1b-9d57-f6a9c6538be4
62640038-b31a-4503-be1e-ace7a20fa397
71a57228-e96a-4c14-a3df-093d14c8f82d
74b6aee1-d39c-48b2-8524-776cae011514
913391df-a13d-44fe-93fa-3ff89e682334
918c32b1-4c50-42f9-bbb5-2fa0ead9a451
919992b5-9001-4168-ac5c-f6b7354e02e8
94dfc8af-6c1d-4a2a-9242-dd4eee271118
bff6563f-4db1-4bfe-890e-0b4fc377a479
c20834ea-f2a7-4fdd-8d6c-cd800687317d
c4a42b03-f1ff-41c5-af11-1354c1066e9d
c6a254b4-3812-490e-a782-09c8d4ec1472
db4f4e84-9892-4af4-961f-2d8ecfb732c2
e0287346-6a87-4593-8601-feb0583d7ea8
e116263b-3600-455f-806e-4f4192aa33cd
e7fdeb9c-9bcd-416a-bc13-7fe793741468
f2d79bd4-9991-4ac1-932f-dadcc2f3a4ed
fa4ce1b5-0522-448b-a7d4-f2fc74f94ace
fe6b2573-998f-4eff-9f2d-a66c927dde4c


In [12]:
# Don't use mice with nans
uuids_all = list(h5_temp.keys())
uuids = copy.deepcopy(uuids_all)
delete_inds=[]
for i in range(len(uuids_all)):
   # print(np.sum(np.isnan(h5_temp[uuids_all[i]][:])))
    if np.sum(np.isnan(h5_temp[uuids_all[i]][:]))>0:
        delete_inds.append(i)
for ii in sorted(delete_inds,reverse=True):
    del uuids[ii]
    
for i in range(n_mice):
   # print(uuids[i])
    if np.sum(np.isnan(h5_temp[uuids[i]][:])) > 0:
        print('ERROR: NANS STILL PRESENT')
        ver


In [13]:
#### CREATE DATA GENERATORS ####

def data_generator(pca_file, uuids, batch_size,n_mice):
    
    pca_file = h5_temp
    
    total_batches=0
    n_batches = [None]*n_mice

    for i_mouse in range(n_mice):
        length_mouse = pca_file[uuids[i_mouse]].shape[0]
        n_batches[i_mouse] = np.floor(length_mouse/batch_size)

    total_batches = int(np.sum(n_batches))

    batch_inds = np.zeros((int(total_batches),2))
    i_pos=0
    
    for i_mouse in range(n_mice):
        for i_batch in range(int(n_batches[i_mouse])):
            batch_inds[i_pos,0] = i_mouse
            batch_inds[i_pos,1] = i_batch
            i_pos+=1

    loop_vec = np.arange(total_batches)
    
    
    for i_epoch in range(1000):
        np.random.shuffle(loop_vec)
        for ii in loop_vec:

            i_mouse = int(batch_inds[ii,0])
            which_batch = int(batch_inds[ii,1])

            yield pca_file[uuids[i_mouse]][which_batch*batch_size:(which_batch+1)*batch_size], i_mouse, which_batch #, behavioral_labels[i_mouse][which_bucket][which_batch*batch_size:(which_batch+1)*batch_size],depth[i_mouse][which_bucket][which_batch*batch_size:(which_batch+1)*batch_size],i_mouse,which_bucket,which_batch


In [14]:
pca_file = h5_temp

total_batches=0
n_batches = [None]*n_mice

for i_mouse in range(n_mice):
    length_mouse = pca_file[uuids[i_mouse]].shape[0]
    n_batches[i_mouse] = np.floor(length_mouse/batch_size)
    
total_batches = int(np.sum(n_batches))
nb_tng_batches = total_batches
print(nb_tng_batches)

901


In [15]:
data_gen = data_generator(h5_temp,uuids,batch_size,n_mice)

In [16]:
def log_sum_exp(value, dim=None, keepdim=False):
    """Numerically stable implementation of the operation
    value.exp().sum(dim, keepdim).log()
    """
    # TODO: torch.max(value, dim=None) threw an error at time of writing
    if dim is not None:
        m, _ = torch.max(value, dim=dim, keepdim=True)
        value0 = value - m
        if keepdim is False:
            m = m.squeeze(dim)
        return m + torch.log(torch.sum(torch.exp(value0),
                                       dim=dim, keepdim=keepdim))
    else:
        m = torch.max(value)
        sum_exp = torch.sum(torch.exp(value - m))
        return m + torch.log(sum_exp)


In [17]:
### Losses

from datta.messages.torchhmm import hmm_marginal_likelihood

def arhmm_log_probability(z, h, log_Ps, As, bs, Qs, nlags):
    """
    Computes p(h | A, C) = \sum_z p(h, z | Ps, As, bs, Qs)
    using message passing algorithm for hidden Markov models.
    
    p(h, z | Ps, As, bs, Qs) = \prod_t N(h[t] | As[z[t]] h[t-1] + bs[z[t]], Qs[z[t]])
                                    \times P[z[t] | z[t-1]]
    
    @param z       T         array of integer valued states 1, ..., K
    @param h       T x H     array of observations
    @param log_Ps: T x K x K set of log transition matrices for each time step t
    @param As:     K x H x H set of dynamics matrices
    @param bs:     K x H     set of dynamics biases
    @param log_Qs: K x H     matrix of (diagonal) log variances of h given z
    """
    T, H = h.shape
    K = log_Ps.shape[1]
    assert log_Ps.shape == (T-1, K, K)
    assert As.shape == (K, H*nlags, H)
    assert bs.shape == (K, H)
    assert Qs.shape == (K, H)
    
    # Initialize output 
    lp = 0
    
    # Assume a uniform initial distribution
    log_pi0 = -math.log(K) * torch.ones(K)

    # Make sure transition matrices are normalized
    log_Ps = log_Ps - log_sum_exp(log_Ps, dim=-1, keepdim=True)

    # Compute log p(theta)
    alpha = 6
    kappa = 1e8
    Ps = torch.exp(log_Ps)
    for i_state in range(K):
        concentration = alpha*torch.ones(K) / K
        concentration[i_state] += kappa
        dirichlet_dist = torch.distributions.dirichlet.Dirichlet(concentration)
        lp += torch.sum(dirichlet_dist.log_prob(Ps[i_state]))
    
    # Compute log p(z | theta)
    lp += torch.sum(log_Ps[torch.range(0, T-2).long(), z[:-1], z[1:]])
    
    # Precompute the log likelihoods
    # N(h[t+1] | As[z[t]] h[t] + bs[z[t]], Qs[z[t]])    
    E_hs = torch.matmul(torch.cat(([h[nlags-1-i:T-1-i] for i in range(nlags,-1,-1)]),dim=1).contiguous().view(T-nlags, 1, 1, H*nlags), As).squeeze(2) + bs       # T-1 x K x H
    E_hs = torch.cat((bs.view(1, K, H).repeat(nlags,1,1) , E_hs), 0)                             # T x K x H

    # Note: you could for-loop over lags and add up the contributions for each one
    #       this would avoid allocating and copying memory
    # E_hs = torch.zeros(T, K, H)
    # E_hs += bs
    # for lag in range(nlags):
    #    ... 
        
    lls = -0.5 * torch.sum((h.unsqueeze(1) - E_hs)**2 / Qs, dim=2)         # T x K
    lls += -0.5 * torch.sum(math.log(2 * math.pi) + torch.log(Qs), dim=1)  # K 

    # Compute log p(x | z, theta)
    lp += torch.sum(lls[torch.range(0, T-1).long(), z])

    return lp

def DeepInputDrivenARHMM_EM_loss(zs, h, log_Ps, As, bs, Qs, nlags=1):
    """
    zs is a (S, T) LongTensor with S samples of the discrete states
    """
    K, D, _ = As.shape
    S, _ = zs.shape
    
    # log prob is likelihood plus prior
    lp = 0
    for s in range(S):
        lp += arhmm_log_probability(zs[s], h, log_Ps, As, bs, Qs, nlags) / S
    
    return -lp / (h.shape[0] * h.shape[1])


In [ ]:
### Losses

from datta.messages.torchhmm import hmm_marginal_likelihood

def arhmm_marginal_likelihood(h, log_Ps, As, bs, Qs, nlags):
    """
    Computes p(h | A, C) = \sum_z p(h, z | Ps, As, bs, Qs)
    using message passing algorithm for hidden Markov models.
    
    p(h, z | Ps, As, bs, Qs) = \prod_t N(h[t] | As[z[t]] h[t-1] + bs[z[t]], Qs[z[t]])
                                    \times P[z[t] | z[t-1]]
    
    @param H       T x H     array of observations
    @param log_Ps: T x K x K set of log transition matrices for each time step t
    @param As:     K x H x H set of dynamics matrices
    @param bs:     K x H     set of dynamics biases
    @param log_Qs: K x H     matrix of (diagonal) log variances of h given z
    """
    T, H = h.shape
    K = log_Ps.shape[1]
    assert log_Ps.shape == (T-1, K, K)
    assert As.shape == (K, H*nlags, H)
    assert bs.shape == (K, H)
    assert Qs.shape == (K, H)

    # Assume a uniform initial distribution
    log_pi0 = -math.log(K) * torch.ones(K)

    # Make sure transition matrices are normalized
    log_Ps = log_Ps - log_sum_exp(log_Ps, dim=-1, keepdim=True)

    # Precompute the log likelihoods
    # N(h[t+1] | As[z[t]] h[t] + bs[z[t]], Qs[z[t]])
    #AsT = As.transpose(1, 2)
    
    E_hs = torch.matmul(torch.cat(([h[nlags-1-i:T-1-i] for i in range(nlags,-1,-1)]),dim=1).contiguous().view(T-nlags, 1, 1, H*nlags), As).squeeze(2) + bs       # T-1 x K x H
    #E_hs = torch.matmul(h[:-1].contiguous().view(T-1, 1, 1, H), As).squeeze(2) + bs       # T-1 x K x H
    E_hs = torch.cat((bs.view(1, K, H).repeat(nlags,1,1) , E_hs), 0)                             # T x K x H
    ## TO DO: padding with nlags of b, should I do something better here, ignore lags or pad with A*what there is
    lls = -0.5 * torch.sum((h.unsqueeze(1) - E_hs)**2 / Qs, dim=2)  # T x K
    lls += -0.5 * torch.sum(math.log(2 * math.pi) + torch.log(Qs), dim=1)            # K 

    return hmm_marginal_likelihood(log_pi0, log_Ps, lls)

def DeepInputDrivenARHMM_loss(h, log_Ps, As, bs, Qs, nlags=1):
    K, D, _ = As.shape
    alpha = 6
    kappa = 1e8
    # Make sure transition matrices are normalized
    log_Ps = log_Ps - log_sum_exp(log_Ps, dim=-1, keepdim=True)
    Ps = torch.exp(log_Ps)
    for i_state in range(K):
        concentration = alpha*torch.ones(K) / K
        concentration[i_state] += kappa
        dirichlet_dist = torch.distributions.dirichlet.Dirichlet(concentration)
        if i_state==0:
            prior_trans = torch.sum(dirichlet_dist.log_prob(Ps[i_state]))
        else:
            prior_trans += torch.sum(dirichlet_dist.log_prob(Ps[i_state]))
    #Qs = torch.ones((K,D)) #torch.eye(D).repeat(K, 1, 1)
    
    # log prob is likelihood plus prior
    log_prob = arhmm_marginal_likelihood(h, log_Ps, As, bs, Qs, nlags) + prior_trans
    
    return -log_prob/(h.shape[0]*h.shape[1])



In [18]:
## Set up model

from datta.messages.torchhmm import hmm_sample

class ARHMM(nn.Module):
    def __init__(self, K, H, kappa, batch_size, nlags):
        super(ARHMM, self).__init__()
        self.n_discrete_states = K
        self.latent_dim_size_h = H
        self.kappa=kappa
        self.batch_size=batch_size
        self.nlags = nlags
        self.__build_model()
    
    def __build_model(self):

        # Emission parameters
        self.As = nn.Parameter(torch.zeros((self.n_discrete_states, self.latent_dim_size_h*self.nlags, self.latent_dim_size_h)))
        self.bs = nn.Parameter(torch.zeros((self.n_discrete_states, self.latent_dim_size_h)))
        #self.inv_softplus_Qs = nn.Parameter(torch.zeros((self.n_discrete_states, self.latent_dim_size_h)))
        self.inv_softplus_Qs = nn.Parameter(torch.ones((self.n_discrete_states, self.latent_dim_size_h)))

        # Transition bias network (x -> transition bias + transition prob)
        self.log_transition_proba = \
                nn.Parameter(torch.log(
                self.kappa * torch.eye(self.n_discrete_states) + (1-self.kappa) / self.n_discrete_states * torch.ones((self.n_discrete_states, self.n_discrete_states))))

        self.softplus = nn.Softplus()

    def forward(self):
        log_Ps = self.log_transition_proba
        log_Ps = log_Ps.unsqueeze(0).repeat(self.batch_size-1,1,1)
        
        return log_Ps, self.As, self.bs, self.softplus(self.inv_softplus_Qs) #E_y, var_y, mu_post, var_post, h, log_Ps, self.As, self.bs, self.softplus(self.inv_softplus_Qs)

    def loss(self, h, log_Ps, As, bs, Qs):
        return DeepInputDrivenARHMM_loss(h, log_Ps, As, bs, Qs, self.nlags) 

    def sample_states(self, h, num_samples=1):
        log_pi0 = ...
        log_Ps = ...
        lls = ...
        
        # Concatenate into an S x T LongTensor
        return torch.cat([hmm_sample(log_pi0, log_Ps, lls) for _ in range(num_samples)])
    
    def EM_loss(self, zs, h, log_Ps, As, bs, Qs):
        return DeepInputDrivenARHMM_EM_loss(zs, h, log_Ps, As, bs, Qs, self.nlags) 

    
    def get_expected_states(self, h, log_Ps, As, bs, Qs):

        T, H = h.shape
        K = log_Ps.shape[1]

        # Get log likelihoods
        # Make sure transition matrices are normalized
        log_Ps = log_Ps - log_sum_exp(log_Ps, dim=-1, keepdim=True)
        log_PsT = log_Ps.transpose(1, 2)

        #AsT = As.transpose(1, 2)
        E_hs = torch.matmul(torch.cat(([h[self.nlags-1-i:T-1-i] for i in range(self.nlags,-1,-1)]),dim=1).contiguous().view(T-self.nlags, 1, 1, H*self.nlags), As).squeeze(2) + bs       # T-1 x K x H
        #E_hs = torch.matmul(h[:-1].contiguous().view(T-1, 1, 1, H), As).squeeze(2) + bs       # T-1 x K x H
        E_hs = torch.cat((bs.view(1, K, H).repeat(self.nlags,1,1) , E_hs), 0)                             # T x K x H
        
       # E_hs = torch.matmul(h[:-1].contiguous().view(T-1, 1, 1, H), As).squeeze(2) + bs       # T-1 x K x H
       # E_hs = torch.cat((bs.view(1, K, H), E_hs), 0)                             # T x K x H
        #lls = -0.5 * torch.sum((h.unsqueeze(1) - E_hs)**2 / Qs, dim=2)  # T x K
        #lls += -0.5 * torch.sum(math.log(2 * math.pi) + torch.log(Qs), dim=1)            # K 
        lls = -0.5 * torch.sum((h.unsqueeze(1) - E_hs)**2 / Qs, dim=2)  # T x K
        lls += -0.5 * torch.sum(math.log(2 * math.pi) + torch.log(Qs), dim=1)            # K 
        
#         lls = torch.zeros(T,K)
#         for i_t in range(T):
#             for i_k in range(K):
#                 normal_dist = torch.distributions.multivariate_normal.MultivariateNormal(E_hs[i_t,i_k],covariance_matrix=Qs[i_k])
#                 lls[i_t,i_k] = normal_dist.log_prob(h[i_t])
                
        # Forwards pass
        alpha = torch.zeros_like(lls)
        alpha[0] = -math.log(self.n_discrete_states) + lls[0] 
        for t in range(alpha.shape[0]-1):
            alpha[t+1] = log_sum_exp(alpha[t] + log_PsT[t],dim=1) + lls[t+1]

        # Backwards pass
        beta = torch.zeros_like(lls)
        for t in range(beta.shape[0]-2,-1,-1):
            beta[t] = log_sum_exp(log_Ps[t]+beta[t+1]+lls[t+1],dim=1)

        # Combine to get posterior over z
        expected_z = alpha+beta
        expected_z -= expected_z.max(1)[0].view((-1,1))
        expected_z = torch.exp(expected_z)
        expected_z /= expected_z.sum(1).view((-1,1))

        return expected_z
    
    def get_expected_states_covmat(self, h, log_Ps, As, bs, Qs):

        T, H = h.shape
        K = log_Ps.shape[1]

        # Get log likelihoods
        # Make sure transition matrices are normalized
        log_Ps = log_Ps - log_sum_exp(log_Ps, dim=-1, keepdim=True)
        log_PsT = log_Ps.transpose(1, 2)

        #AsT = As.transpose(1, 2)
        E_hs = torch.matmul(torch.cat(([h[self.nlags-1-i:T-1-i] for i in range(self.nlags,-1,-1)]),dim=1).contiguous().view(T-self.nlags, 1, 1, H*self.nlags), As).squeeze(2) + bs       # T-1 x K x H
        #E_hs = torch.matmul(h[:-1].contiguous().view(T-1, 1, 1, H), As).squeeze(2) + bs       # T-1 x K x H
        E_hs = torch.cat((bs.view(1, K, H).repeat(self.nlags,1,1) , E_hs), 0)                             # T x K x H
        
       # E_hs = torch.matmul(h[:-1].contiguous().view(T-1, 1, 1, H), As).squeeze(2) + bs       # T-1 x K x H
       # E_hs = torch.cat((bs.view(1, K, H), E_hs), 0)                             # T x K x H
        #lls = -0.5 * torch.sum((h.unsqueeze(1) - E_hs)**2 / Qs, dim=2)  # T x K
        #lls += -0.5 * torch.sum(math.log(2 * math.pi) + torch.log(Qs), dim=1)            # K 
        #lls = -0.5 * torch.sum((h.unsqueeze(1) - E_hs)**2 / Qs, dim=2)  # T x K
        #lls += -0.5 * torch.sum(math.log(2 * math.pi) + torch.log(Qs), dim=1)            # K 
        
        lls = torch.zeros(T,K)
        for i_t in range(T):
            for i_k in range(K):
                normal_dist = torch.distributions.multivariate_normal.MultivariateNormal(E_hs[i_t,i_k],covariance_matrix=Qs[i_k])
                lls[i_t,i_k] = normal_dist.log_prob(h[i_t])
                
        # Forwards pass
        alpha = torch.zeros_like(lls)
        alpha[0] = -math.log(self.n_discrete_states) + lls[0] 
        for t in range(alpha.shape[0]-1):
            alpha[t+1] = log_sum_exp(alpha[t] + log_PsT[t],dim=1) + lls[t+1]

        # Backwards pass
        beta = torch.zeros_like(lls)
        for t in range(beta.shape[0]-2,-1,-1):
            beta[t] = log_sum_exp(log_Ps[t]+beta[t+1]+lls[t+1],dim=1)

        # Combine to get posterior over z
        expected_z = alpha+beta
        expected_z -= expected_z.max(1)[0].view((-1,1))
        expected_z = torch.exp(expected_z)
        expected_z /= expected_z.sum(1).view((-1,1))

        return expected_z
 
    def initialize_transitions(self, data_gen, nb_tng_batches, L2_reg=0.01):

        As = torch.zeros((self.n_discrete_states, self.latent_dim_size_h*self.nlags, self.latent_dim_size_h))
        bs = torch.zeros((self.n_discrete_states, self.latent_dim_size_h))
        Qs = torch.zeros((self.n_discrete_states, self.latent_dim_size_h))

        data_split = np.floor(nb_tng_batches/self.n_discrete_states)
        
        # Initialize
        i_discrete_state = 0
        XTX = Variable(torch.zeros((self.latent_dim_size_h*self.nlags+1,self.latent_dim_size_h*self.nlags+1)))
        XTY = Variable(torch.zeros((self.latent_dim_size_h*self.nlags+1,self.latent_dim_size_h)))
        
        start_collecting=1

        for batch_nb in range(nb_tng_batches):

            pca_score, i_mouse, i_batch = next(data_gen)
            pca_score = torch.autograd.Variable(torch.from_numpy(pca_score)).float()
            X = torch.cat(([pca_score[nlags-1-i:pca_score.shape[0]-1-i] for i in range(nlags,-1,-1)]),dim=1) #pca_score[0:-1]
            X = F.pad(X,(1,0),value=1)
            Y = pca_score[self.nlags:]

            XTX += torch.matmul(X.transpose(1,0),X)
            XTY += torch.matmul(X.transpose(1,0),Y)

            if start_collecting:
                all_X = X
                all_Y = Y
                start_collecting=0
            else:
                all_X = torch.cat((all_X,X),0)
                all_Y = torch.cat((all_Y,Y),0)
            if i_discrete_state < self.n_discrete_states:
                if np.mod(batch_nb+1,data_split)==0:
                    reg_matrix = L2_reg*Variable(torch.eye(X.shape[1]))
                    reg_XTX = XTX+reg_matrix
                    XTX_inv = torch.inverse(reg_XTX)
                    W = torch.matmul(XTX_inv,XTY)

                    As[i_discrete_state] = W[1:,:].data
                    bs[i_discrete_state] = W[0,:].data

                    # Reconstruct to get residuals 
                    Y_hat = torch.matmul(all_X,W)
                    residuals = Y_hat-all_Y
                    Qs[i_discrete_state] = torch.var(residuals,0).data

                    start_collecting=1

                    i_discrete_state +=1
                    XTX = Variable(torch.zeros((self.latent_dim_size_h*self.nlags+1,self.latent_dim_size_h*self.nlags+1)))
                    XTY = Variable(torch.zeros((self.latent_dim_size_h*self.nlags+1,self.latent_dim_size_h)))
        if i_discrete_state < self.n_discrete_states-1:
            print('ERROR WITH INITIALIZATION')
            sys.exit(0)

        inv_softplus_Qs = torch.log(torch.exp(Qs)-1)
        return As, bs, inv_softplus_Qs


In [19]:
kappa=0.9
K=100
H=10
nlags=3
model = ARHMM(K,H,kappa,batch_size,nlags)

In [20]:
## Initialize model
As, bs, inv_softplus_Qs = model.initialize_transitions(data_gen, nb_tng_batches)
model.As.data = As
model.bs.data = bs
model.inv_softplus_Qs.data = inv_softplus_Qs

In [21]:
# Look at initialization autoregressive forward pass results
# T = 230
# i_z = 20
# h = np.zeros((T, 10))
# h[0] = bs[i_z].numpy()
# Qs = np.log(np.exp(inv_softplus_Qs.numpy())+1)
# for t in range(T-1):
#     h[t+1] = np.dot(As[i_z].numpy().T,h[t])+bs[i_z].numpy() + Qs[i_z]**(1/2)*np.random.normal(size=(10,))

# plt.plot(h);


In [22]:
#log_Ps, As, bs, Qs = model()


In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [26]:
from tqdm import tqdm

In [27]:
####### Analysis of their model ########

# ar_mat = np.load('ar_mat.npy')
# sig = np.load('sig.npy')
# transition_matrix = np.load('transition_matrix.npy')
import joblib

model_fit = 'clean_data_pca_gibbs_sampling_100_iter.p'
model_fit = joblib.load(model_fit)

kappa=0.9
K=100
H=10
nlags=3


model2 = ARHMM(K,H,kappa,batch_size,nlags)


model2.As.data = torch.tensor(np.swapaxes(np.asarray(model_fit['model_parameters'][0]['ar_mat'])[:,:,:-1],2,1)).float()
model2.bs.data = torch.tensor(np.asarray(model_fit['model_parameters'][0]['ar_mat'])[:,:,-1]).float()
sig = np.asarray(model_fit['model_parameters'][0]['sig'])
diag_sig = np.asarray([np.diag(sig[i]) for i in range(100)])
Qs = torch.tensor(sig).float()
model2.inv_softplus_Qs.data = torch.tensor(np.log(np.exp(diag_sig)-1)).float()
model2.log_transition_proba.data = torch.tensor(np.log(np.asarray(model_fit['model_parameters'][0]['transition_matrix']))).float()



In [28]:
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-2)
for i_epoch in range(1):
    train_loss=0
    for i_train in tqdm(range(nb_tng_batches)):
        pca, i_mouse, i_batch = next(data_gen)
        pca = torch.tensor(pca).to(device).float()
        #pca_torch = torch.autograd.Variable(torch.from_numpy(pca)).float()
        optimizer.zero_grad()
        loss = model2.loss(pca,*model2())
        if i_epoch > 0:
            loss.backward()
            optimizer.step()
        train_loss+=loss.item()
    print("Epoch: ", i_epoch, " ", train_loss/nb_tng_batches)


100%|██████████| 901/901 [07:09<00:00,  2.10it/s]

Epoch:  0   2705177088.0


In [29]:
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=.01)
for i_epoch in range(100):
    train_loss=0
    for i_train in tqdm(range(nb_tng_batches)):
        pca, i_mouse, i_batch = next(data_gen)
        pca = torch.tensor(pca).to(device).float()
        #pca_torch = torch.autograd.Variable(torch.from_numpy(pca)).float()
        optimizer.zero_grad()
        loss = model.loss(pca,*model())
        if i_epoch > 0:
            loss.backward()
            optimizer.step()
        train_loss+=loss.item()
    print("Epoch: ", i_epoch, " ", train_loss/nb_tng_batches)


  0%|          | 0/901 [00:00<?, ?it/s]

Epoch:  0   683965312.0


  2%|▏         | 14/901 [00:56<59:44,  4.04s/it]

KeyboardInterrupt: 

In [ ]:
# NEW! Train with the EM loss
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=.01)
for i_epoch in range(100):
    train_loss = 0
    for i_train in tqdm(range(nb_tng_batches)):
        pca, i_mouse, i_batch = next(data_gen)
        pca = torch.tensor(pca).to(device).float()
        #pca_torch = torch.autograd.Variable(torch.from_numpy(pca)).float()
        
        zs = model.sample_states(pca)
        
        optimizer.zero_grad()
        loss = model.EM_loss(zs, pca, *model())
        if i_epoch > 0:
            loss.backward()
            optimizer.step()
        train_loss += loss.item()
    print("Epoch: ", i_epoch, " ", train_loss/nb_tng_batches)


In [ ]:
plt.imshow(np.exp(model.log_transition_proba.detach().numpy()))
plt.colorbar()

In [ ]:
plt.imshow(model.As[4].detach().numpy())

In [ ]:
####### Analysis of their model ########

# ar_mat = np.load('ar_mat.npy')
# sig = np.load('sig.npy')
# transition_matrix = np.load('transition_matrix.npy')
import joblib

model_fit = 'clean_data_pca_gibbs_sampling_100_iter.p'
model_fit = joblib.load(model_fit)

kappa=0.9
K=100
H=10
nlags=3


model = ARHMM(K,H,kappa,batch_size,nlags)


model.As.data = torch.tensor(np.swapaxes(np.asarray(model_fit['model_parameters'][0]['ar_mat'])[:,:,:-1],2,1)).float()
model.bs.data = torch.tensor(np.asarray(model_fit['model_parameters'][0]['ar_mat'])[:,:,-1]).float()
sig = np.asarray(model_fit['model_parameters'][0]['sig'])
diag_sig = np.asarray([np.diag(sig[i]) for i in range(100)])
Qs = torch.tensor(sig).float()
model.inv_softplus_Qs.data = torch.tensor(np.log(np.exp(diag_sig)-1)).float()
# np.log(np.asarray(model_fit['model_parameters'][0]['transition_matrix']
model.log_transition_proba.data = torch.tensor(np.log(np.asarray(model_fit['model_parameters'][0]['transition_matrix']))).float()




In [ ]:
transition_matrix = torch.tensor(np.asarray(model_fit['model_parameters'][0]['transition_matrix'])).float()

In [ ]:
plt.imshow(transition_matrix.detach().numpy())
plt.colorbar()

In [ ]:
transition_matrix[20,0]

In [ ]:
np.sum(transition_matrix[10].detach().numpy())

In [ ]:
alpha=6
kappa=1e8
i_state=1
concentration = alpha*torch.ones(K)
concentration[i_state] += kappa
dirichlet_dist = torch.distributions.dirichlet.Dirichlet(concentration)
#dirichlet_dist = torch.distributions.dirichlet.Dirichlet(transition_matrix[i_state])
dirichlet_dist.log_prob(transition_matrix[i_state]) #/K

In [ ]:
import scipy.stats
scipy.stats.dirichlet.logpdf(transition_matrix[i_state].detach().numpy(),concentration.detach().numpy()) #/100

In [ ]:
scipy.stats.dirichlet.logpdf(transition_matrix[i_state].detach().numpy(),transition_matrix[i_state].detach().numpy())

In [ ]:
np.exp(0)

In [ ]:
K=100
for i_state in range(K):
    concentration = alpha*torch.ones(K)
    concentration[i_state] += kappa
    dirichlet_dist = torch.distributions.dirichlet.Dirichlet(concentration)
    if i_state==0:
        prior_trans = torch.sum(dirichlet_dist.log_prob(Ps[i_state]))
    else:
        prior_trans += torch.sum(dirichlet_dist.log_prob(Ps[i_state]))

In [ ]:
alpha=6
kappa=1e8
K=100
i_state = 1

concentration = alpha*torch.ones(K)
concentration[i_state] += kappa
dirichlet_dist = torch.distributions.dirichlet.Dirichlet(concentration)
-dirichlet_dist.log_prob(transition_matrix[i_state])

In [ ]:

pca, i_mouse, which_batch = next(data_gen)
pca = torch.tensor(pca).to(device).float()
model2 = ARHMM(K,H,kappa,batch_size,nlags)

loss = model.loss(pca,*model())
loss2 = model2.loss(pca,*model2())

In [ ]:
loss

In [ ]:
loss2

In [ ]:
pca, i_mouse, which_batch = next(data_gen)
pca = torch.tensor(pca).to(device).float()
expected_z = model.get_expected_states(pca,*model())


In [ ]:
plt.imshow(expected_z.data.numpy().T[:,0:300],origin='lower',cmap='gray_r')


In [ ]:
pca, i_mouse, which_batch = next(data_gen)
pca = torch.tensor(pca).to(device).float()
log_Ps, As, bs, not_Qs = model()

expected_z = model.get_expected_states_covmat(pca,log_Ps, As, bs,Qs)
real_labels = model_fit['labels'][0][model_fit['keys'].index(uuids[i_mouse])].reshape((-1,))[which_batch*batch_size:(which_batch+1)*batch_size]


In [ ]:
plt.imshow(expected_z.data.numpy().T[:,0:300],origin='lower',cmap='gray_r')
#plt.plot(np.argmax(expected_z.data.numpy(),1)[0:500],'r')
plt.plot(real_labels[:300],'r',alpha=.2)
plt.ylim([0,105])
plt.colorbar()
plt.xlabel('Time')
plt.ylabel('Discrete state')
#plt.savefig('gibbs_states.png')

In [ ]:
plt.plot(real_labels)
#plt.plot(np.argmax(expected_z.data.numpy(),1),'r',alpha=.5)
plt.xlim([0,300])

In [ ]:
plt.plot(np.argmax(expected_z.data.numpy(),1))
plt.xlim([0,500])

In [ ]:
plt.imshow((model.log_transition_proba+Variable(6 * torch.eye(model.log_transition_proba.shape[0]))).data.numpy())
plt.colorbar()

In [ ]:
plt.imshow(np.exp((model.log_transition_proba+Variable(6 * torch.eye(model.log_transition_proba.shape[0]))).data.numpy()))
plt.colorbar()

In [ ]:
rr = np.load('keys.npy')

In [ ]:
model_fit